In [1]:
import torch
import torch.nn as nn

# Embedding

Embeddings take a tensor of indices of tokens, into a matrix of the embeddings of the tokens. 

In [24]:
vocab_size = 5
embedding_dim = 7

emb = torch.nn.Embedding(vocab_size, embedding_dim)

emb

Embedding(5, 7)

In [27]:
x = torch.tensor([0,1,2,3,4,4,4,4])
print(x.size())

embeddings = emb(x)
print(embeddings.size())
print(embeddings)

torch.Size([8])
torch.Size([8, 7])
tensor([[-0.9485, -0.6305, -0.7886,  2.6438,  1.0426,  0.0383,  0.6728],
        [-0.3032,  1.0501, -0.6864, -2.0268, -1.1542,  0.2204, -0.7051],
        [ 0.9960,  0.1500,  0.6221,  1.2173,  0.9628, -0.1682, -1.3474],
        [-0.6563,  0.5737,  1.1994,  0.7263,  0.1355, -0.4578, -0.7757],
        [ 0.0100,  1.9865, -0.4506,  0.8747,  0.4641,  0.4897, -0.3365],
        [ 0.0100,  1.9865, -0.4506,  0.8747,  0.4641,  0.4897, -0.3365],
        [ 0.0100,  1.9865, -0.4506,  0.8747,  0.4641,  0.4897, -0.3365],
        [ 0.0100,  1.9865, -0.4506,  0.8747,  0.4641,  0.4897, -0.3365]],
       grad_fn=<EmbeddingBackward0>)


In [28]:
# what if x is batched input? 
x_batched = torch.tensor([[0,1,2,2], [2,3,4,4]])
print(x_batched.size())

embeddings = emb(x_batched)
print(embeddings.size())
print(embeddings)

torch.Size([2, 4])
torch.Size([2, 4, 7])
tensor([[[-0.9485, -0.6305, -0.7886,  2.6438,  1.0426,  0.0383,  0.6728],
         [-0.3032,  1.0501, -0.6864, -2.0268, -1.1542,  0.2204, -0.7051],
         [ 0.9960,  0.1500,  0.6221,  1.2173,  0.9628, -0.1682, -1.3474],
         [ 0.9960,  0.1500,  0.6221,  1.2173,  0.9628, -0.1682, -1.3474]],

        [[ 0.9960,  0.1500,  0.6221,  1.2173,  0.9628, -0.1682, -1.3474],
         [-0.6563,  0.5737,  1.1994,  0.7263,  0.1355, -0.4578, -0.7757],
         [ 0.0100,  1.9865, -0.4506,  0.8747,  0.4641,  0.4897, -0.3365],
         [ 0.0100,  1.9865, -0.4506,  0.8747,  0.4641,  0.4897, -0.3365]]],
       grad_fn=<EmbeddingBackward0>)


In [32]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        # self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        # self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [33]:
model = LSTMTagger(10, 20, 30, 40)

print(model)

LSTMTagger(
  (word_embeddings): Embedding(30, 10)
)


In [35]:
for param in model.parameters(): 
    print(param)
    print(param.size())

Parameter containing:
tensor([[-2.5813e-01, -6.4703e-01, -9.1433e-01, -8.1346e-01, -1.0241e+00,
         -2.8796e-01,  3.4297e-02,  8.0998e-01, -1.3215e+00,  1.1946e+00],
        [ 1.0358e+00, -5.4781e-01, -7.6452e-01,  1.6602e-01,  3.3481e-01,
         -5.9183e-01,  1.0333e+00, -8.0865e-01,  3.4254e-01,  1.3579e+00],
        [-2.8722e-01, -7.2576e-02, -5.3387e-01,  7.6659e-01,  5.0026e-01,
         -8.2063e-02, -3.2224e-02, -1.6364e+00, -3.3055e-02, -8.8303e-01],
        [-1.1859e+00, -2.4185e-01,  4.2906e-01,  2.7880e-01,  1.9719e-02,
          1.0389e+00,  1.1196e+00,  1.0873e+00, -7.1420e-01,  4.9449e-01],
        [ 1.5089e+00,  1.9056e+00,  6.8070e-01,  1.7093e-01, -6.0116e-01,
         -1.4972e+00,  4.3371e-01, -3.8173e-01, -5.8957e-01,  1.0867e+00],
        [ 2.8598e-02,  4.2503e-01,  8.3138e-02,  1.5146e-02, -1.5358e+00,
          3.4631e-01, -1.1374e+00,  8.8088e-01, -7.5425e-02, -2.1819e+00],
        [ 9.7590e-01, -8.5094e-01,  1.4902e+00,  9.2025e-01, -5.9858e-01,
          

# Loss

## Example of target with class indices

In [59]:

loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)


In [60]:
input.size()

torch.Size([3, 5])

In [61]:
target.size()

torch.Size([3])

In [62]:
input

tensor([[-0.3556, -1.0022, -0.7583, -0.5866, -1.5448],
        [ 2.2494,  1.0764,  1.2176, -0.5809, -0.1928],
        [ 0.9878, -0.0733,  1.5109,  0.4811, -0.4841]], requires_grad=True)

In [63]:
input.grad

In [64]:
target

tensor([2, 1, 1])

In [65]:
output.backward()

In [68]:
input.grad

tensor([[ 0.1013,  0.0531, -0.2656,  0.0804,  0.0308],
        [ 0.1840, -0.2764,  0.0656,  0.0109,  0.0160],
        [ 0.0862, -0.3035,  0.1455,  0.0520,  0.0198]])

## Example of target with class probabilities

In [77]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)

In [78]:
input.size()

torch.Size([3, 5])

In [79]:
target.size()

torch.Size([3, 5])

In [80]:
input

tensor([[ 0.2569,  2.2694,  0.4131, -1.1755, -0.7490],
        [-0.9746, -1.0044, -0.7197,  0.2863,  0.6351],
        [-0.4826, -1.0022,  0.1887,  0.0504, -0.1648]], requires_grad=True)

In [81]:
input.grad

In [82]:
target

tensor([[0.1581, 0.1017, 0.0918, 0.3352, 0.3132],
        [0.2083, 0.5078, 0.1210, 0.0648, 0.0981],
        [0.0972, 0.0098, 0.2173, 0.0755, 0.6001]])

In [83]:
output.backward()

In [84]:
input.grad

tensor([[-0.0202,  0.2093,  0.0074, -0.1040, -0.0925],
        [-0.0412, -0.1418, -0.0039,  0.0782,  0.1087],
        [ 0.0179,  0.0266,  0.0259,  0.0605, -0.1310]])

In [50]:
output.grad

/tmp/ipykernel_127885/250565712.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400268359/work/build/aten/src/ATen/core/TensorBody.h:489.)
  output.grad


# tensor

In [2]:
import torch

# Your 2D tensor
tensor_2d = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Get the last row
last_row = tensor_2d[-1]

print(last_row)

tensor([7, 8, 9])


In [4]:
len(tensor_2d.size())

2

In [5]:
h_0 = torch.zeros(256)

h_0.size()

torch.Size([256])

In [6]:
h_0 = h_0.unsqueeze(0).unsqueeze(0)

h_0.size()

torch.Size([1, 1, 256])

# extract the last non-zero along an axis

In [47]:
def get_sample_tensor(non_zeros, zeros): 
    t = torch.arange(non_zeros)

    t0 = torch.zeros(zeros, dtype=t.dtype)

    result_tensor = torch.cat((t, t0)).view(8,4)
    return result_tensor

In [48]:
t1 = get_sample_tensor(20, 12)
t2 = get_sample_tensor(32, 0)
t3 = get_sample_tensor(16, 16)

t1.size()

torch.Size([8, 4])

In [51]:
rnn_output = torch.stack([t1, t2, t3], dim=0)
rnn_output.size()

torch.Size([3, 8, 4])

In [52]:
rnn_output

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0]],

        [[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0],
         [ 0,  0,  0,  0]]])

In [53]:
# Sample sequence lengths (L)
sequence_lengths = [5, 8, 4]

In [86]:
# Create a mask to indicate valid positions
mask = torch.arange(rnn_output.size(1)).unsqueeze(0) < torch.tensor(sequence_lengths).unsqueeze(1)
mask


tensor([[ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True, False, False, False, False]])

In [57]:
# Find the last non-zero position along the sequence dimension
last_non_zero_positions = torch.max(mask * torch.arange(rnn_output.size(1)), dim=1).values
last_non_zero_positions

tensor([4, 7, 3])

In [92]:
mask * torch.arange(rnn_output.size(1))

tensor([[0, 1, 2, 3, 4, 0, 0, 0],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 0, 0, 0, 0]])

In [58]:
# Extract the last non-zero tensor along the sequence dimension
final_outputs = rnn_output[torch.arange(rnn_output.size(0)), last_non_zero_positions]

print(final_outputs.shape)  # Size: (N, output_size)

torch.Size([3, 4])


In [99]:
# temp 
torch.stack([torch.tensor(1),torch.tensor(2)])

tensor([1, 2])

In [100]:
torch.tensor(1).size()

torch.Size([])

## broadcasting

In the case of comparing a tensor of size (1, 8) and a tensor of size (3, 1), the result will be a tensor of size (3, 8). This is again due to broadcasting rules.

When performing element-wise operations (such as comparison) between two tensors, PyTorch compares their dimensions element-wise, **starting from the rightmost dimension**. Broadcasting allows the tensors to be compatible for element-wise operations when:

1. The size of the dimensions matches.
2. One of the sizes is 1.  

Let's look at the example:

Tensor A: (1, 8)
Tensor B: (3, 1)  

In this case, PyTorch will broadcast the smaller tensor (Tensor B) along its singleton dimension to match the size of the corresponding dimension in the larger tensor (Tensor A). Broadcasting will stretch Tensor B to have the same size as Tensor A along the second dimension. As a result, you get a tensor of size (3, 8) when performing element-wise operations.

In [90]:
mask = torch.arange(rnn_output.size(1)).unsqueeze(0) < torch.tensor(sequence_lengths).unsqueeze(1)
mask

tensor([[ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True, False, False, False, False]])

In [87]:
# temp
print(torch.arange(rnn_output.size(1)))
print(torch.arange(rnn_output.size(1)).size())
print(torch.arange(rnn_output.size(1)).unsqueeze(0))
print(torch.arange(rnn_output.size(1)).unsqueeze(0).size())

tensor([0, 1, 2, 3, 4, 5, 6, 7])
torch.Size([8])
tensor([[0, 1, 2, 3, 4, 5, 6, 7]])
torch.Size([1, 8])


In [89]:
# temp
print(torch.tensor(sequence_lengths))
print(torch.tensor(sequence_lengths).size())
print(torch.tensor(sequence_lengths).unsqueeze(1))
print(torch.tensor(sequence_lengths).unsqueeze(1).size())

tensor([5, 8, 4])
torch.Size([3])
tensor([[5],
        [8],
        [4]])
torch.Size([3, 1])


# Extract specific element along axis

In [109]:
# Create a tensor of size (3, 10)
tensor = torch.arange(30).view(3, 10)
tensor


tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]])

In [115]:
seq_lengths = torch.tensor([2,4,8])
seq_lengths

tensor([2, 4, 8])

In [118]:
indices = seq_lengths - 1
indices

tensor([1, 3, 7])

In [119]:
# Use fancy indexing to extract elements from each row
result = tensor[range(tensor.size(0)), indices]

print(result)


tensor([ 1, 13, 27])


In [120]:
result = tensor[[0,1,2], indices]
result

tensor([ 1, 13, 27])

In [123]:
# indices = (seq_lengths - 1).view(-1, 1).expand(len(seq_lengths), out_unpacked.size(2)).unsqueeze(0)
indices = (seq_lengths - 1).view(-1, 1).expand(len(seq_lengths))
indices

RuntimeError: expand(torch.LongTensor{[3, 1]}, size=[3]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)

# pack_padded_sequence

In [101]:
# seq_lengths: tensor([43,  35,  30, 138])
# sorted_indices=tensor([3, 0, 1, 2]), unsorted_indices=tensor([1, 2, 3, 0])

# [138, 43, 35, 30]

# for the first 30 cells, all 4 tensor have it
# continue till the 35th cell, the first 3 tensors have it
# continue till the 43th cell, the first 2 tensors have it
# till the end, only the first 1 tensor have it

batch_sizes = [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [102]:
len([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4])

30

In [103]:
len([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3])

35

In [104]:
len([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2])

43

In [106]:
len([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

138

In [107]:
sum([138, 43, 35, 30])

246

In [63]:
torch.arange(8).size()

torch.Size([8])

In [61]:
torch.arange(8)[None, :]

tensor([[0, 1, 2, 3, 4, 5, 6, 7]])

In [64]:
torch.arange(8)[None, :].size()

torch.Size([1, 8])

In [108]:
len([ 0.2054,  0.0020,  0.0486,  0.0508,  0.1717,  0.4226,  0.0641,  0.2693,
        -0.1672,  0.5834,  0.1972,  0.0074,  0.2665, -0.1019, -0.2332, -0.2746,
         0.2326,  0.2747,  0.1856,  0.2407,  0.4252,  0.2461,  0.2270,  0.4674,
        -0.3185, -0.3030, -0.1261,  0.0362, -0.0045,  0.0117,  0.1414,  0.1806,
         0.3911,  0.4545, -0.2965,  0.0517,  0.1228,  0.4499,  0.4384,  0.0195,
         0.1236, -0.1288,  0.0365,  0.0000])

44

# sort a counter 

In [20]:
tensor_str1 = """
(tensor([ 113, 1376,  519, 1320,   10,    7, 1811, 1338, 1148,    7, 4311,    3,
         113, 4312,   24, 1676, 6154,  821, 2084,   25,    0, 6155, 4312,    3,
        1148,    3,  112,    3, 1811,  951,  803, 3720, 1123, 2084,    0,    6,
           8,    2,    9]), tensor(1))
"""

In [21]:
tensor_str2 = """
(tensor([ 113, 1376,  519, 1320,   10,    7, 1811, 1338, 1148,    7, 4311,    3,
         113, 4312,   24, 1676, 6154,  821, 2084,   25,    0, 6155, 4312,    3,
        1148,    3,  112,    3, 1811,  951,  803, 3720, 1123, 2084,    0,    6,
           8,    2,    9]), tensor(1))
"""

In [22]:
print(tensor_str1 == tensor_str2)

True


# End